# GitHubリポジトリのスクレイピング

In [9]:
import requests
from bs4 import BeautifulSoup
import time
import sqlite3

## 設定

In [10]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

base_url = 'https://github.com/orgs/google/repositories'
db_name = 'github_repos.db'


## データベースの作成

In [11]:
try:
    conn = sqlite3.connect(db_name)
    cur = conn.cursor()
    
    sql = '''CREATE TABLE IF NOT EXISTS repositories (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                language TEXT,
                stars TEXT
            );'''
    
    cur.execute(sql)
    conn.commit()
    
except sqlite3.Error as e:
    print('エラーが発生しました:', e)
else:
    print('テーブルの作成が完了しました。')
finally:
    conn.close()


テーブルの作成が完了しました。


## スクレイピング関数

In [12]:
def scrape_google_repos(max_pages=3):
    repos = []
    page = 1
    
    while page <= max_pages:
        url = f"{base_url}?page={page}"
        
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            
        except requests.exceptions.HTTPError as e:
            print("HTTPエラーが発生しました:", e)
            break
        except requests.exceptions.RequestException as e:
            print("リクエストエラーが発生しました:", e)
            break
        else:
            soup = BeautifulSoup(response.text, 'html.parser')
            repo_items = soup.select('li[data-hovercard-type="repository"]')
            
            if not repo_items:
                break
            
            for item in repo_items:
                try:
                    name_elem = item.select_one('h3 a')
                    name = name_elem.text.strip() if name_elem else "Unknown"
                    
                    lang_elem = item.select_one('[itemprop="programmingLanguage"]')
                    language = lang_elem.text.strip() if lang_elem else "None"
                    
                    star_elem = item.select_one('a[href*="stargazers"]')
                    stars = star_elem.text.strip() if star_elem else "0"
                    
                    repos.append((name, language, stars))
                except:
                    continue
            
            time.sleep(1)
            page += 1
        finally:
            print(f"ページ {page-1} 完了")
    
    return repos


## スクレイピング実行

In [13]:
repos = scrape_google_repos(max_pages=3)
print(f"{len(repos)}件取得")

ページ 0 完了
0件取得


## データベースへの保存

In [14]:
try:
    conn = sqlite3.connect(db_name)
    cur = conn.cursor()
    
    sql = "INSERT INTO repositories (name, language, stars) VALUES (?, ?, ?);"
    cur.executemany(sql, repos)
    conn.commit()
    
except sqlite3.Error as e:
    print('エラーが発生しました:', e)
else:
    print(f'{len(repos)}件保存完了')
finally:
    conn.close()


0件保存完了


## データベースの表示

In [15]:
try:
    conn = sqlite3.connect(db_name)
    cur = conn.cursor()
    
    sql = "SELECT * FROM repositories;"
    cur.execute(sql)
    
except sqlite3.Error as e:
    print('エラーが発生しました:', e)
else:
    for row in cur:
        id, name, language, stars = row
        print(f"{id} | {name} | {language} | {stars}")
finally:
    conn.close()
